## Load the dataset

In [1]:
import pandas as pd
import numpy as np
import pickle
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.model_selection import PredefinedSplit
from sklearn import preprocessing

save_in_test_folder = True
if save_in_test_folder:
    filepath = "../TestModule"
else:
    filepath = "."

seed = 42
FILENAME = "dataset/train_dataset.csv"

#Prepare train data
df1 = pd.read_csv(FILENAME, sep=",", low_memory=False)

# get features names
features = list(df1.columns)
features_to_remove = ["label", "ts", "src_ip", "dst_ip", "dns_query", "ssl_subject", "ssl_issuer", "http_uri", "type", "http_referrer", "http_user_agent"]
features = [feature for feature in features if feature not in features_to_remove]
df1 = df1[features + ["type"]]

# Converte i valori in numeri, sostituendo quelli non validi con NaN
df1["src_bytes"] = pd.to_numeric(df1["src_bytes"], errors='coerce')
# Filtra le righe con NaN (valori non convertibili)
df1 = df1.dropna(subset=["src_bytes"])
# Converte i valori rimasti in interi
df1.loc[:, "src_bytes"] = df1["src_bytes"].astype(int)

print("#Righe: " + str(df1.shape[0]) + " #Colonne: " + str(df1.shape[1]))
df1 = df1.dropna()
print("#Righe: " + str(df1.shape[0]) + " #Colonne: " + str(df1.shape[1]))

X = df1[features]
y = df1["type"]

le = preprocessing.LabelEncoder()
le.fit(y)
with open(f"{filepath}/transformer/target_encoder.save", "wb") as f:
    pickle.dump(le, f)

y = le.transform(y)

indices = np.arange(X.shape[0])
train_idx, val_idx = train_test_split(indices, test_size=0.2, stratify=y, random_state=seed)

# fold = np.zeros(X.shape[0])
# fold[train_idx] = -1

n_splits = 61
skf = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=42)

fold = np.full(len(y), -1)  # Inizializza tutto con -1 (default: train)

# Assegna i fold ai campioni
for fold_number, (_, val_idx) in enumerate(skf.split(X, y)):
    fold[val_idx] = fold_number  # Assegna il numero del fold ai campioni di validazione


ps = PredefinedSplit(fold)
ps.get_n_splits()

# for i, (train_index, test_index) in enumerate(ps.split()):
#     print(f"Fold {i}:")
#     print(f"  Train: index={train_index}")
#     print(f"  Test:  index={test_index}")

# take only x with index in val_idx
X_val = X.iloc[val_idx]
y_val = y[val_idx]
X_train = X.iloc[train_idx]
y_train = y[train_idx]

#Righe: 616983 #Colonne: 37
#Righe: 616983 #Colonne: 37


In [2]:
df1

,src_port,dst_port,proto,service,duration,src_bytes,dst_bytes,conn_state,missed_bytes,src_pkts,...,http_request_body_len,http_response_body_len,http_status_code,http_user_agent,http_orig_mime_types,http_resp_mime_types,weird_name,weird_addl,weird_notice,type
0,53972,10502,tcp,-,0.000000,0.0,0,OTH,0,0,...,0,0,0,-,-,-,-,-,-,normal
1,37513,53,udp,dns,0.163608,47.0,423,SF,0,1,...,0,0,0,-,-,-,-,-,-,normal
2,2077,2077,tcp,-,0.208218,0.0,0,S0,0,120,...,0,0,0,-,-,-,-,-,-,normal
3,53972,10502,tcp,-,0.000000,0.0,0,OTH,0,0,...,0,0,0,-,-,-,-,-,-,normal
4,1880,47979,tcp,-,0.000000,0.0,0,OTH,0,1,...,0,0,0,-,-,-,-,-,-,normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
616997,53116,53,udp,dns,0.044893,84.0,424,SF,0,2,...,0,0,0,-,-,-,-,-,-,mitm
616998,57669,53,udp,dns,0.002957,84.0,436,SF,0,2,...,0,0,0,-,-,-,-,-,-,mitm
616999,54730,53,udp,dns,0.016624,58.0,178,SF,0,2,...,0,0,0,-,-,-,-,-,-,mitm
617000,59846,443,tcp,ssl,48.271568,3219.0,1212,SF,0,26,...,0,0,0,-,-,-,-,-,-,mitm


## Preprocess the dataset

In [2]:
from sklearn.preprocessing import MinMaxScaler, LabelEncoder, OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.impute import KNNImputer, SimpleImputer

import pickle

categorical_columns = X_train.select_dtypes(include=["object"]).columns.tolist()
numeric_columns = X_train.select_dtypes(include=["int64", "float64"]).columns.tolist()
# boolean_columns = X_train.select_dtypes(include=["bool"]).columns.tolist()

ct = ColumnTransformer(
    [
        ("cat", OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1), categorical_columns),  # Trasforma le colonne categoriche
        # ("ordinal", OneHotEncoder(handle_unknown='infrequent_if_exist', sparse_output=False), categorical_columns),  # Trasforma le colonne categoriche
        ("scale", StandardScaler(), numeric_columns)  # Normalizza le colonne numeriche
    ],
    remainder="passthrough"  # Mantieni le altre colonne invariate
)
ct.set_output(transform="pandas")

ct = ct.fit(X_train)
with open(f"{filepath}/transformer/transformer.save", "wb") as f:
    pickle.dump(ct, f)

# train set
X_train = ct.transform(X_train)

# validation set
X_val = ct.transform(X_val)

# X
X = ct.transform(X)

In [ ]:
from sklearn import decomposition
from sklearn import discriminant_analysis

# pca = decomposition.PCA(n_components=0.95, random_state=seed)
pca = discriminant_analysis.LinearDiscriminantAnalysis(n_components=9)

pca.set_output(transform="pandas")

pca.fit(X_train, y_train)
with open(f"{filepath}/transformer/pca.save", "wb") as f:
    pickle.dump(pca, f)

X_train_pca = pca.transform(X_train)
X_val_pca = pca.transform(X_val)
X_pca = pca.transform(X)

# X_train = X_train_pca
# X_val = X_val_pca
# X = X_pca

## Apply K-Nearest Neighbour

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import RandomizedSearchCV
import os

os.environ['OMP_NUM_THREADS'] = '4'
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
    'n_neighbors': [3, 5, 7, 9, 11],  # Number of neighbors to consider
    'weights': ['uniform', 'distance'],  # Weight function
    'metric': ['euclidean', 'manhattan', 'minkowski'],  # Distance metrics
    'p': [1, 2],  # Minkowski parameter (1 for Manhattan, 2 for Euclidean)
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute']  # Algorithm to compute neighbors
}

grid = RandomizedSearchCV(KNeighborsClassifier(), param_grid, cv=ps, n_iter=2, random_state=seed)
grid.fit(X, y)

print("Best parameters:", grid.best_params_)
print("Best score:", grid.best_score_)
best_knn = grid.best_estimator_
with open(f"{filepath}/models/knn.save", "wb") as file:
    pickle.dump(best_knn, file)

pd.DataFrame(grid.cv_results_)


# knn = KNeighborsClassifier().fit(X_train, y_train)
# print("Performance:", knn.score(X_val, y_val))
# with open( f"{filepath}/models/knn.save", "wb") as file:
#     pickle.dump(knn, file)

- Performance: 0.9187280941672238 con minmax
- Performance: 0.9549626207986386 senza minmax

## Apply Random Forest

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

param_grid = {
    'n_estimators': [100, 200, 300, 500],  # Number of trees in the forest
    'max_depth': [None, 10, 20, 30, 50],  # Maximum depth of the tree
    'min_samples_split': [2, 5, 10],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required at a leaf node
    'max_features': ['sqrt', 'log2', None],  # Number of features to consider when looking for the best split
    'bootstrap': [True, False],  # Whether bootstrap samples are used when building trees
    'criterion': ['gini', 'entropy', 'log_loss'],  # Split quality measure
    'class_weight': ['balanced', 'balanced_subsample', None]  # Weights associated with classes
}

grid = RandomizedSearchCV(RandomForestClassifier(random_state=seed), param_grid, cv=ps, n_iter=2, random_state=seed)
grid.fit(X, y)

print("Best parameters:", grid.best_params_)
print("Best score:", grid.best_score_)
best_rf = grid.best_estimator_
with open(f"{filepath}/models/rf.save", "wb") as file:
    pickle.dump(best_rf, file)

pd.DataFrame(grid.cv_results_)

rf = RandomForestClassifier(random_state=seed, class_weight="balanced").fit(X_train, y_train)
print("Performance:", rf.score(X_val, y_val))
with open(f"{filepath}/models/rf.save", "wb") as file:
    pickle.dump(rf, file)

ValueError: could not convert string to float: 'tcp'

## Apply Support Vector Classifier with HP tuning

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from skopt import BayesSearchCV

param_grid = {'C': [0.1, 1, 10, 100, 1000],
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf']}

# param_grid = {
#     'C': (0.1, 100, 'log-uniform'),
#     'gamma': (0.001, 10, 'log-uniform'),
#     'kernel': ["rbf"]
# }

grid = BayesSearchCV(
    SVC(random_state=seed),
    param_grid,
    n_iter=1,  # Numero massimo di iterazioni
    cv=ps,
    verbose=1,
    random_state=seed,
    n_jobs=6
)
grid.fit(X, y)

# grid = RandomizedSearchCV(SVC(random_state=seed), param_grid, cv=ps)
# grid.fit(X, y)
#
print("Best parameters:", grid.best_params_)
print("Best score:", grid.best_score_)
best_svm = grid.best_estimator_
with open(f"{filepath}/models/svm.save", "wb") as file:
    pickle.dump(best_svm, file)

pd.DataFrame(grid.cv_results_)

# svm = SVC(random_state=seed, class_weight="balanced", verbose=1, C=100).fit(X_train.head(10000), y_train[:10000])
# print("Performance:", svm.score(X_val, y_val))
# with open(f"{filepath}/models/svm.save", "wb") as file:
#     pickle.dump(svm, file)



Fitting 61 folds for each of 1 candidates, totalling 61 fits


KeyboardInterrupt: 